In [1]:
import pandas as pd
import numpy as np
import datetime 
import os 

In [2]:
#read in the latest dataset
time_series = pd.read_csv("../../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,145,145,145,145,149,149,151,151,151,151
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,15,15,15,16,16,19
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1757,1808,1843,1924,1968,2031,2156,2173,2198,2256
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,66,66,66,66,68,68,69,69


In [3]:
#keep only rows with NC county data
time_series = time_series[time_series["Province_State"] == "North Carolina"]
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
1895,84037001,US,USA,840,37001.0,Alamance,North Carolina,US,36.043470,-79.399761,...,121,124,128,128,128,143,149,152,173,183
1896,84037003,US,USA,840,37003.0,Alexander,North Carolina,US,35.922380,-81.177519,...,4,4,4,5,6,8,8,10,12,12
1897,84037005,US,USA,840,37005.0,Alleghany,North Carolina,US,36.493609,-81.128570,...,6,6,6,6,7,7,7,7,8,8
1898,84037007,US,USA,840,37007.0,Anson,North Carolina,US,34.974032,-80.099533,...,29,30,30,30,31,33,37,38,38,38
1899,84037009,US,USA,840,37009.0,Ashe,North Carolina,US,36.432962,-81.498627,...,5,5,5,5,5,6,7,8,13,13


In [4]:
#define a dataframe with just the county information
nc_counties = time_series[["FIPS", "Admin2","Province_State","Lat","Long_"]]
nc_counties.head()

,FIPS,Admin2,Province_State,Lat,Long_
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570
1898,37007.0,Anson,North Carolina,34.974032,-80.099533
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627


In [5]:
#define a variable to define the end date of data to be included
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

122

In [6]:
#define a dataframe with just the total cases by county each day
dates = time_series.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
1895,0,0,0,0,0,0,0,0,0,0,...,121,124,128,128,128,143,149,152,173,183
1896,0,0,0,0,0,0,0,0,0,0,...,4,4,4,5,6,8,8,10,12,12
1897,0,0,0,0,0,0,0,0,0,0,...,6,6,6,6,7,7,7,7,8,8
1898,0,0,0,0,0,0,0,0,0,0,...,29,30,30,30,31,33,37,38,38,38
1899,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,6,7,8,13,13


In [7]:
#define an empty dataframe to transpose the cases by date into a new row for each county-date pair
nc_time_series = pd.DataFrame(columns = ["FIPS","Admin2","Province_State","Lat","Long_","Confirmed","Date"])

In [8]:
#fill in the dataframe
cols = dates.columns

for i in range(14,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Date"] = cols[i]
    county_time = nc_counties.join(one_day)
    nc_time_series = nc_time_series.append(county_time)
#    county_time.append(one_day)
#    print(one_day)
nc_time_series.head()


,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Date
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761,0,3/18/20
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519,0,3/18/20
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570,0,3/18/20
1898,37007.0,Anson,North Carolina,34.974032,-80.099533,0,3/18/20
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627,0,3/18/20


In [9]:
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)

In [10]:
nc_time_series["Date"] = pd.to_datetime(nc_time_series.Date)
nc_time_series.dtypes

FIPS                     float64
Admin2                    object
Province_State            object
Lat                      float64
Long_                    float64
Confirmed                 object
Date              datetime64[ns]
dtype: object

In [11]:
#convert Date column to format that will be uniform across use cases
nc_time_series["Date"] = nc_time_series["Date"].dt.to_period("D")

In [12]:
nc_time_series.tail()

,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Date
5605,37195.0,Wilson,North Carolina,35.706255,-77.917978,216,2020-05-11
5606,37197.0,Yadkin,North Carolina,36.162226,-80.662394,54,2020-05-11
5607,37199.0,Yancey,North Carolina,35.900792,-82.312372,9,2020-05-11
5608,80037.0,Out of NC,North Carolina,0.000000,0.000000,0,2020-05-11
5609,90037.0,Unassigned,North Carolina,0.000000,0.000000,0,2020-05-11


In [13]:
nc_time_series.to_excel("../clean_data/nc_time_series.xlsx")